In [ ]:
# This code is for checking 
# Preprocessing method for 법정동

In [ ]:
from glob import glob

import pandas as pd
import geopandas as gpd

In [ ]:
bdong_shp_paths = glob("./data/raw_data/법정동_좌표/*/*.shp")
bdong_shp_paths

In [ ]:
def read_gdf(file_path, encoding):
    gdf = gpd.read_file(file_path, encoding=encoding)
    return gdf


def gdf2df(gdf):
    return pd.DataFrame(gdf)


read_gdf_with_cp949 = lambda file_path: read_gdf(file_path, "cp949")
read_shp = lambda file_path: gdf2df(read_gdf_with_cp949(file_path))

In [ ]:
bdong_shp_df = pd.concat([read_shp(shp_path) for shp_path in bdong_shp_paths])
bdong_shp_df.head()

In [ ]:
def read_csv(file_path, params):
    return pd.read_csv(file_path, **params)

read_csv_params = {
    "dtype": {"시도코드": str, "시군구코드": str, "법정동코드": str},
}

read_csv_with_params = lambda file_path: read_csv(file_path, read_csv_params)

In [ ]:
file_path = "./data/preproc_data/bdong_raw.csv"
bdong_base = read_csv_with_params(file_path)
bdong_base.head()

In [ ]:
bdong_base["rough_name"] = bdong_base["시도명"] + "_" + bdong_base["시군구명"].fillna("")
rough_code_name_dict = bdong_base.set_index("시군구코드")['rough_name'].to_dict()

In [ ]:
bdong_shp_df["rough_name"] = bdong_shp_df["COL_ADM_SE"].astype(str).map(rough_code_name_dict)

In [ ]:
preproc_bdong_shp_df = bdong_shp_df.loc[:,['EMD_NM','EMD_CD','rough_name','geometry']]
preproc_bdong_shp_df.columns = ["detail_name", "detail_code", "rough_name", "geometry"]